In [1]:
import streamlit as st
import pandas as pd
import plotly.express as px
import geopandas as gpd
import folium
from folium import Choropleth
from streamlit_folium import folium_static
import openai
import requests
import os
from dotenv import load_dotenv

st.header("Mapa da Evasão - Proporção de evadidos")
st.write("Mapa da Evasão no Brasil")
    # Adicione o código relevante aqui

    # Configuração da página no Streamlit
    # st.set_page_config(page_title="Mapa de Evadidos", layout="wide")

    # Carregar datasets
data_path = '../../artifacts/microdados_eficiencia_academica_RedeFederal_2023_tecnico_RegiaoMetropolitana.csv'
data_geo = pd.read_csv(data_path, sep=';')


2025-05-26 15:54:46.906 
  command:

    streamlit run c:\Users\jabsondias\AppData\Local\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [2]:
codUF_path = '../../artifacts/codigo_estados.csv'
data_cod = pd.read_csv(codUF_path, sep=';', encoding='ISO-8859-1')


In [8]:
 # Carregar shapefile dos estados do Brasil (GeoJSON)
geojson_path = '../../artifacts/BR_UF_2024.geojson'
gdf_estados = gpd.read_file(geojson_path)

In [ ]:
codUF_path = '../../artifacts/codigo_estados.csv'
data_cod = pd.read_csv(codUF_path, sep=';', encoding='ISO-8859-1')

    # Converter colunas para string
data_geo['UF'] = data_geo['UF'].astype(str)
data_cod['SG_UF'] = data_cod['SG_UF'].astype(str)

    # Merge entre dados e códigos de UF
data_geo_merge = pd.merge(
        data_geo, data_cod, left_on='UF', right_on='SG_UF', how='left')

    # Total de matrículas por UF
total_por_uf = data_geo_merge.groupby(
        'NM_UF').size().reset_index(name='total_matriculas')

    # Total de evadidos por UF
evadidos = data_geo_merge[data_geo_merge['CATEGORIA_SITUACAO'] == 'Evadidos']
evadidos_por_uf = evadidos.groupby(
        'NM_UF').size().reset_index(name='evadidos')

    # Merge para calcular proporção
proporcao_df = pd.merge(
        total_por_uf, evadidos_por_uf, on='NM_UF', how='left')
proporcao_df['evadidos'] = proporcao_df['evadidos'].fillna(0)
proporcao_df['proporcao'] = (
    proporcao_df['evadidos'] / proporcao_df['total_matriculas']) * 100

    # Carregar shapefile dos estados do Brasil (GeoJSON)
geojson_path = '../../artifacts/pages/BR_UF_2024.geojson'
gdf_estados = gpd.read_file(geojson_path)

    # Dissolver para obter geometria por estado
gdf_estados = gdf_estados.dissolve(by='NM_UF', as_index=False)

    # Merge com dados de proporção
gdf_mapa = gdf_estados.merge(proporcao_df, on='NM_UF', how='left')
gdf_mapa['proporcao'] = gdf_mapa['proporcao'].fillna(0)

    # Centro aproximado do Brasil
latitude_centro = -14.2350
longitude_centro = -51.9253

    # Criar mapa base
mapa = folium.Map(location=[
                    latitude_centro, longitude_centro], tiles="Cartodb Positron", zoom_start=5)

    # Mapa coroplético com proporção
Choropleth(
        geo_data=gdf_estados,
        data=gdf_mapa,
        columns=['NM_UF', 'proporcao'],
        key_on='properties.NM_UF',
        fill_color='YlOrRd',
        nan_fill_color='white',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name='Proporção de Evadidos (%) por Estado'
).add_to(mapa)

    # Estilo para destaque
def estilo(x): return {"fillColor": "white",
                           "color": "black", "fillOpacity": 0.001, "weight": 0.001}

def estilo_destaque(x): return {
        "fillColor": "darkblue", "color": "black", "fillOpacity": 0.5, "weight": 1}

    # Adicionar tooltip com proporção
highlight = folium.features.GeoJson(
        data=gdf_mapa,
        style_function=estilo,
        highlight_function=estilo_destaque,
        tooltip=folium.features.GeoJsonTooltip(
            fields=['NM_UF', 'proporcao'],
            aliases=['Estado:', 'Proporção de Evadidos (%):'],
            localize=True,
            style=("background: white; color: black"),
            labels=True,
            sticky=True
        )
)
highlight.add_to(mapa)

    # Controles
folium.LayerControl().add_to(mapa)

    # Exibir
folium_static(mapa, width=1400, height=800)

DataSourceError: Failed to read GeoJSON data